In [1]:
import json
import numpy as np
import os
import re
import nltk

In [2]:
words = set(nltk.corpus.words.words())

In [3]:
stopwords=['','(',')','{','}','\\','--',':']

In [4]:
tweets=[]
for line in open("20110123-000.json", 'r'):
    try:
        tweets.append(json.loads(line))
    except:
        continue
        
tweets[14]        

{'contributors': None,
 'coordinates': None,
 'created_at': 'Sun Jan 23 00:00:01 +0000 2011',
 'favorited': False,
 'geo': None,
 'id': 28965135238303744,
 'id_str': '28965135238303744',
 'in_reply_to_screen_name': 'BeasBookNook',
 'in_reply_to_status_id': 28962147899805696,
 'in_reply_to_status_id_str': '28962147899805696',
 'in_reply_to_user_id': 47618028,
 'in_reply_to_user_id_str': '47618028',
 'place': None,
 'retweet_count': 0,
 'retweeted': False,
 'source': '<a href="http://www.tweetdeck.com" rel="nofollow">TweetDeck</a>',
 'text': "@BeasBookNook every ARC I've ever read hasn't had final copy edits. Isn't that the nature of the beast? I guess some are better than others.",
 'truncated': False,
 'user': {'contributors_enabled': False,
  'created_at': 'Tue Apr 03 18:05:30 +0000 2007',
  'default_profile': False,
  'default_profile_image': False,
  'description': 'Truth be told, I think I thrive under a lack of accountability. /Michael Scott',
  'favourites_count': 0,
  'follow_re

In [5]:
test_tweets= []
for line in open("20110123-006.json", 'r'):
    try:
        test_tweets.append(json.loads(line))
    except:
        continue

# Making language model:

In [6]:
from nltk.util import ngrams
import string
from nltk import SimpleGoodTuringProbDist
from math import log

In [7]:
corpus = nltk.corpus.reader.plaintext.PlaintextCorpusReader(".", "sample.txt")
test = nltk.corpus.reader.plaintext.PlaintextCorpusReader(".", "test_data.txt")
print(test)
corpus

<PlaintextCorpusReader in 'C:\\Users\\Sudhanshu\\Downloads\\twitter_bot'>


<PlaintextCorpusReader in 'C:\\Users\\Sudhanshu\\Downloads\\twitter_bot'>

## Unigram perplexity with add 1 smoothing\ Laplacian smoothing

In [8]:
unigram= [w.strip(string.punctuation).lower() for w in corpus.words()]
len(unigram)
#unigram[:20]

43784

In [16]:
spl= int(95*len(unigram)/100)
train_corpus= unigram[:spl]
test_corpus= unigram[spl:]

In [18]:
freq_1gram= nltk.FreqDist(train_corpus)
len_train= len(train_corpus)
vocab= len(set(train_corpus))

In [19]:
def unigram_prob_with_add1smoothing(word):
    return (freq_1gram[word]+1)/(len_train+vocab)

In [20]:
def entropy(n, text):
        e = 0.0
        text = ["<s>"] + text + ["</s>"]
        for i in range(n - 1, len(text)):
            context = text[i - n + 1:i]
            token = text[i]
            #print(str(context)+"    "+token)
            e += logprob(token, context)
        return e / float(len(text) - (n - 1))


def logprob(word, context):
    if len(context)==0:
        p=unigram_prob_with_add1smoothing(word)
    elif len(context)==1:
        p=bigram_prob_with_add1smoothing(context[0], word)
    else:
        p=trigram_prob_with_add1smoothing(context[0], context[1], word)
    return -p*log(p , 2)


def perplexity(n, text):
      return pow(2.0, entropy(n, text))

In [27]:
perplexity(1, test_corpus)

1.024138089875898

In [28]:
perplexity(1, train_corpus)

1.025133170322091

## Bigram Perplexity with add 1 smoothing

In [9]:
bigram= [a for a in ngrams(unigram, 2, pad_left=True,pad_right=True,left_pad_symbol='<s>', right_pad_symbol="</s>")]
len(bigram)

43785

In [26]:
cfreq_2gram = nltk.ConditionalFreqDist(bigram)
cprob_2gram = nltk.ConditionalProbDist(cfreq_2gram, nltk.MLEProbDist)
def bigram_prob_with_add1smoothing(word1, word2):
    cprob_2gram_add1=float((((1+cfreq_2gram[word1][word2])/(len(cfreq_2gram)+sum(cfreq_2gram[word1].values())))))
    return cprob_2gram_add1


In [30]:
perplexity(2, test_corpus)

1.0081485615099057

In [29]:
perplexity(2, train_corpus)

1.0085341354101802

In [40]:
def bigram_prob_with_gtsmoothing(word1, word2):
    return SimpleGoodTuringProbDist(word1, word2)

In [41]:
def entropy(n, text):
        e = 0.0
        text = ["<s>"] + text + ["</s>"]
        for i in range(n - 1, len(text)):
            context = text[i - n + 1:i]
            token = text[i]
            #print(str(context)+"    "+token)
            e += logprob_gt(token, context)
        return e / float(len(text) - (n - 1))


def logprob_gt(word, context):
    if len(context)==0:
        p=unigram_prob_with_gtsmoothing(word)
    elif len(context)==1:
        p=bigram_prob_with_gtsmoothing(context[0], word)
    else:
        p=trigram_prob_with_gtsmoothing(context[0], context[1], word)
    return -p*log(p , 2)


def perplexity(n, text):
      return pow(2.0, entropy(n, text))

## Trigram perplexity with add 1 smoothing

In [10]:
trigram= [a for a in ngrams(unigram, 3, pad_left= True, pad_right=True, left_pad_symbol= "<s>", right_pad_symbol="</s>")]
len(trigram)

43786

In [32]:
trigrams_as_bigrams=[]
trigrams_as_bigrams.extend([((t[0],t[1]), t[2]) for t in trigram])
cfreq_3gram = nltk.ConditionalFreqDist(trigrams_as_bigrams)
cprob_3gram = nltk.ConditionalProbDist(cfreq_3gram, nltk.MLEProbDist)
def trigram_prob_with_add1smoothing(w1, w2, w3):
    cprob_3gram_add1=float((((1+cfreq_3gram[(w1,w2)][w3])/(len(cfreq_3gram)+sum(cfreq_3gram[(w1,w2)].values())))))
    return cprob_3gram_add1

In [33]:
perplexity(3, test_corpus)

1.0007427952147212

In [34]:
perplexity(3, train_corpus)

1.000768607503031

## Trying to generate text from corpus:

In [17]:
from collections import Counter, OrderedDict, defaultdict
from operator import itemgetter
import random

In [12]:
test_unigram = [w.strip(string.punctuation).lower() for w in test.words()]

test_bigram= [a for a in ngrams(test_unigram, 2)]
len(test_bigram)

test_trigram= [a for a in ngrams(test_unigram, 3)]
len(test_trigram)

test_frequency= nltk.FreqDist(test_bigram)
test_sorted_freq= OrderedDict(reversed(sorted(test_frequency.items(), key= itemgetter(1))))

test_cfreq_2gram= nltk.ConditionalFreqDist(test_bigram)

test_cprob_2gram= nltk.ConditionalProbDist(test_cfreq_2gram, nltk.MLEProbDist)

In [13]:
counts = Counter(corpus.words())
total_count = len(corpus.words())

In [14]:
counts.most_common(20)

[('I', 1173),
 ('a', 1152),
 ('to', 940),
 ('the', 913),
 ('you', 652),
 ('de', 560),
 ('me', 537),
 ('in', 463),
 ('is', 427),
 ('s', 420),
 ('it', 415),
 ('my', 413),
 ('and', 407),
 ('of', 374),
 ('on', 373),
 ('no', 348),
 ('for', 344),
 ('t', 305),
 ('that', 291),
 ('o', 290)]

In [15]:
for word in counts:
    counts[word] /= float(total_count)

In [16]:
# generating 100 words at random of the language
text=[]
for _ in range(100):
    r = random.random()
    accumulator = .0
 
    for word, freq in counts.items():
        accumulator += freq
 
        if accumulator >= r:
            text.append(word)
            break
print(' '.join(text))

bien to Beck many in Messing same you t don China t candy day the here want ser color everywhere I chain anda tan only the follow de swag min june ko knock monte now to and de dis can Proper bien de an mano is we think the have Raise get stupid you DaDa think o match by of die eu This have that ask best our Q em I SOLO shopping re but t that weekly as now pic selling Yo Broken amor I fam BONG de take best de D para outside r fail feel a file


In [23]:
model = defaultdict(lambda: defaultdict(lambda : 0))
for sentence in corpus.sents():
    for w1, w2, w3 in ngrams(sentence, 3, pad_left= True, pad_right=True):
        model[(w1, w2)][w3] +=1

#transform counts to probabilities
for w1_w2 in model:
    total_count = float(sum(model[w1_w2].values()))
    for w3 in model[w1_w2]:
        model[w1_w2][w3] /= total_count


In [ ]:
text = [None, None]
sentence_finished = False
while not sentence_finished:
    r= random.random()
    accumulator = 0.0
    for word in model[tuple(text[-2:])].keys():
        accumulator += model[tuple(text[-2:])][word]
        if(accumulator > r):
            text.append(word)
            break
        if(text[-2:] == [None, None]):
            sentence_finished = True
            
print(" ".join([t for t in text if t]))            